In [0]:
# Classyfing YouTube videos using tags

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from tag_handler import *

In [64]:
from cvs_reader import *
from vocab import *
from dataframe_creator import create_seperate_columns, create_listed_columns

# Get dataframe with tags and category_id columns
ca_csvpath = '/content/CAvideos.csv'
us_csvpath = '/content/USvideos.csv'

tab_dataframe = [load_dataframe(ca_csvpath, ['tags', 'category_id'])]
tab_dataframe.append(load_dataframe(us_csvpath, ['tags', 'category_id']))
# tab_dataframe = pd.Merge(tab_dataframe[0], tab_dataframe[1])

ParserError: ignored

In [61]:
tab_dataframe

,0
0,...
1,...


In [0]:
max_of_tags = 25

vocab = get_tags_vocab(tab_dataframe['tags'])
tab_dataframe = create_seperate_columns(tab_dataframe, max_of_tags, 'tags')

tab_dataframe.head()

In [0]:
dfs = [x for _, x in tab_dataframe.groupby('category_id')]


In [0]:
# Convert vocab to dictionary
voc_di = get_tags_vocab_as_dict(vocab)

# Mapping tags to indexes in vocab
for col in tab_dataframe.columns:
    if col == 'category_id':
      continue
    tab_dataframe[col] = tab_dataframe[col].map(voc_di)

tab_dataframe.fillna(0, inplace = True) 
tab_dataframe.head()

,category_id,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_15,tag_16,tag_17,tag_18,tag_19,tag_20,tag_21,tag_22,tag_23,tag_24
0,10,100346,65978,63363,60526,10870,111791,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,23,21171,94922,23387,141002,127121,13149,112502,81681,1738,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,23,56944,40380,94867,17562,16806,114574,1902,72985,46137,...,88228,55854,136838,47831,119109,136546,88107,82912,-1,-1
3,24,47228,134834,31481,124390,139937,37794,10074,26801,144928,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,10,110425,22894,17422,38563,97956,145924,147907,132840,108881,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [0]:
future_df = []
col_row_dict = {}
# We only use 1/4 of the dataset!
first_half_df = tab_dataframe[:10000]

In [0]:
for i in range(0, 10000):
    for col in first_half_df.columns:
        if col == 'category_id':
            col_row_dict.update({'category_id':first_half_df[col][i]})
        else:
            col_row_dict.update({'tag_{}'.format(first_half_df[col][i]):1.0})
    future_df.append(col_row_dict)
    col_row_dict = {}

In [0]:
final_dataframe = pd.DataFrame(future_df)
final_dataframe.fillna(0.0, inplace = True)
from input_generator import load_dataset_with_lables
final_dataframe.head()

E0817 14:46:21.177545  8056 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\miniconda3\envs\youtube-classification\lib\site-packages\IPython\core\interactiveshell.py", line 3243, in run_ast_nodes
    mod = Module([node], [])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\miniconda3\envs\youtube-classification\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\miniconda3\envs\youtube-classification\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "d:\miniconda3\envs\youtube-classification\lib\site-packages\IPython\core\ultratb.py", line 319, in wrapped
    return f(*ar

KeyboardInterrupt: 

In [0]:
from input_generator import load_dataset_with_lables


# Create train,validation and test examples from tags and labels dataframe
(train_ds, train_lb), (val_ds, val_lb), (test_ds, test_lb) = load_dataset_with_lables(final_dataframe, 'category_id')

In [0]:
train_ds.values

In [0]:
train_lb.values

In [0]:
hidden = 128
model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(hidden,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(hidden,activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(2*hidden,activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(44, activation='softmax')])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )

history = model.fit(train_ds.values, train_lb.values,
                    batch_size=32,
                    epochs=6,
                    validation_data=(val_ds.values, val_lb.values))


In [0]:
import matplotlib.pyplot as plt

history_dict = history.history
acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

plt.plot(acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [0]:
results = model.evaluate(test_ds, test_lb)

print(results)

In [0]:
predictions = model.predict(test_ds)
cat_tit_dict = get_category_title_dict('/content/CA_category_id.json')


In [0]:
def one_hot_to_index(one_hot):
    i_tags = []
    for i, b in enumerate(one_hot):
        if b == 1.0:
        i_tags.append(i)      
    return i_tags

def index_to_tag(i_tags):
w_tags = []
reversed_vocab_dict = {i: vo for vo, i in voc_di.items()}
for index in i_tags:
    w_tags.append(reversed_vocab_dict[index])
return w_tags


In [0]:
for i in range(30, 40):
    one_hot = one_hot_to_index(test_ds.values[i])
    tags_i = index_to_tag(one_hot)
    print(tags_i)
    print('predict:{}'.format(cat_tit_dict[str(np.argmax(predictions[i]))]))
    print('true label:{}'.format(cat_tit_dict[str(test_lb.values[i])]))

# Classifying using tags, number of likes and number of comments


In [0]:
full_dataframe = load_dataframe(csvpath, ['tags', 'likes', 'comment_count', 'category_id'])
max_of_tags = 25

full_dataframe = create_seperate_columns(full_dataframe, max_of_tags, 'tags')

full_dataframe.head()